# Filtered or Hybrid Search Notebook
The purpose of this notebook is to demonstrated the usage of hybrid search and filtering in Astra. 

I'm using a dataset (not very good, though) with support tickets. This work is base on the blog [Large-language models for automatic cloud incident management](https://www.microsoft.com/en-us/research/blog/large-language-models-for-automatic-cloud-incident-management/) 
by Microsoft that proposes adapting large-language models for automated incident management.

In [2]:
# install required dependencies
! pip install --upgrade \
    "cassandra-driver>=3.28.0" \
    "openai>=0.28" \
    "tiktoken==0.4.0" \
    "langchain>=0.0.218" \
    "cassio==0.1.1"

  Obtaining dependency information for openai>=0.28 from https://files.pythonhosted.org/packages/1e/9f/385c25502f437686e4aa715969e5eaf5c2cb5e5ffa7c5cdd52f3c6ae967a/openai-0.28.1-py3-none-any.whl.metadata
  Using cached openai-0.28.1-py3-none-any.whl.metadata (11 kB)
  Obtaining dependency information for langchain>=0.0.218 from https://files.pythonhosted.org/packages/ab/75/262c3e01208c27068144eb76bdf668fad8be97283febaa44f9395ece288b/langchain-0.0.305-py3-none-any.whl.metadata
Using cached openai-0.28.1-py3-none-any.whl (76 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.301
    Uninstalling langchain-0.0.301:
      Successfully uninstalled langchain-0.0.301


In [3]:
# Imports 

import os
import pandas as pd

from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider


In [4]:
# Credential and API keys environment

cass_user = ''
cass_pw = ''
scb_path = 'secure-connect-vector-search-db.zip'

os.environ['OPENAI_API_KEY'] = ""

In [5]:
# Creating a AstraDB Session

cloud_config= {
  'secure_connect_bundle': scb_path
}
auth_provider = PlainTextAuthProvider(cass_user, cass_pw)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()
session.set_keyspace('vsearch')
session

# Data Model

In [80]:
# only use this to reset the schema
session.execute(f"""DROP INDEX IF EXISTS ticket_subject_idx""")
session.execute(f"""DROP INDEX IF EXISTS ticket_opened_idx""")
session.execute(f"""DROP INDEX IF EXISTS embedding_idx""")
session.execute(f"""DROP INDEX IF EXISTS ticket_description_idx""")
session.execute(f"""DROP TABLE IF EXISTS vsearch.vs_rca_learning""")

In [81]:
# # Create Table
session.execute(f"""CREATE TABLE IF NOT EXISTS vsearch.vs_rca_learning
(ticket_id int PRIMARY KEY,
 ticket_opened timestamp,               
 email text,
 product_purchased text,
 ticket_type text,
 ticket_subject text,
 ticket_description text,
 csat_rating float,
 embedding vector<float, 384>);""")

indexes = [
    "CREATE CUSTOM INDEX IF NOT EXISTS ticket_opened_idx ON vsearch.vs_rca_learning (ticket_opened) USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'",
    """
    CREATE CUSTOM INDEX IF NOT EXISTS embedding_idx ON vsearch.vs_rca_learning(embedding)
    USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'
    WITH OPTIONS = { 'similarity_function': 'dot_product' }""",
    """
    CREATE CUSTOM INDEX IF NOT EXISTS ticket_description_idx ON vsearch.vs_rca_learning(ticket_description)
    USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'
    WITH OPTIONS = {
    'index_analyzer': '{
    "tokenizer" : {"name" : "standard"},
    "filters" : [{"name" : "porterstem"},{"name" : "lowercase",	"args": {}}]
    }'};"""

]

for index in indexes:
  session.execute(index)


In [6]:
# Download the all-MiniLM-L6-v2 from HuggingFace, and configure it to infer on the CPU (as opposed to GPU)
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2', device='cpu')


In [7]:
# Loading dataset
df = pd.read_csv("./data/customer_support_tickets.csv")

# Replacing this '{product_purchased}' marker from the Ticket Description column
df["Ticket Description"]= df.apply(lambda x: x['Ticket Description'].replace('{product_purchased}', x['Product Purchased']), axis=1)


# datetime_object = datetime.strptime(datetime_str, '%m/%d/%y %H:%M:%S')
df["First Response Time"]= pd.to_datetime(df["First Response Time"])

# Creating an embedding column with ticket subject and description
df['embedding'] = df['Ticket Subject'] + " " + df['Ticket Description']

# Removing tickets without a RCA
mask = df['Time to Resolution'].notnull()
new_df = df[mask]

new_df.head()


,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating,embedding
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical issue,Network problem,I'm facing a problem with my Dell XPS. The Del...,Closed,Case maybe show recently my computer follow.,Low,Social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0,Network problem I'm facing a problem with my D...
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing inquiry,Account access,I'm having an issue with the Microsoft Office....,Closed,Try capital clearly never color toward story.,Low,Social media,2023-06-01 07:29:40,2023-06-01 01:57:40,3.0,Account access I'm having an issue with the Mi...
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing inquiry,Data loss,I'm having an issue with the Autodesk AutoCAD....,Closed,West decision evidence bit.,Low,Email,2023-06-01 00:12:42,2023-06-01 19:53:42,1.0,Data loss I'm having an issue with the Autodes...
10,11,Joseph Moreno,mbrown@example.org,48,Male,Nintendo Switch,2021-01-19,Cancellation request,Data loss,I'm having an issue with the Nintendo Switch. ...,Closed,Measure tonight surface feel forward.,High,Phone,2023-06-01 17:46:49,2023-05-31 23:51:49,1.0,Data loss I'm having an issue with the Nintend...
11,12,Brandon Arnold,davisjohn@example.net,51,Male,Microsoft Xbox Controller,2021-10-24,Product inquiry,Software bug,I'm having an issue with the Microsoft Xbox Co...,Closed,Measure there house management pick knowledge ...,High,Chat,2023-06-01 12:05:51,2023-06-01 09:27:51,1.0,Software bug I'm having an issue with the Micr...


In [8]:
#If our largest column size is small enough, we don't need to worry about chunking
max_size_desc = new_df['Ticket Description'].apply(lambda x: len(str(x))).max()
print(max_size_desc)

387


In [ ]:
# This block calculates the embedding and loads the rows from the dataframe
insert_pstmt = session.prepare("""
INSERT INTO vsearch.vs_rca_learning
(ticket_id, csat_rating, email, embedding, product_purchased, ticket_description, ticket_opened, ticket_subject, ticket_type)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
""")

#Note that this is a naive loading technique and will be too slow for large data
#For "real" data loads, always use an async approach
for id, row in new_df.iterrows():
  emb = model.encode(row['embedding']).tolist()
  session.execute(insert_pstmt, [row['Ticket ID'], row['Customer Satisfaction Rating'], row['Customer Email'], emb, row['Product Purchased'], row['Ticket Description'], row['First Response Time'], row['Ticket Subject'], row['Ticket Type']])


# Vector Search Testing

In [10]:
#Metadata Filtering example:
#Here, a semantic query is combined with term search and SAI range predicates
#Any non-vector, non-full-text column in the table is metadata for the vector.
new_embed = "Why excel is not working?"
query_vec = model.encode(new_embed).tolist()
hybrid_query = session.execute(
    f"""
    SELECT product_purchased, ticket_opened, ticket_type, ticket_subject, ticket_description
    FROM vsearch.vs_rca_learning
    WHERE ticket_opened > '2023-06-01'
    AND ticket_description : 'loss'
    ORDER BY embedding ANN OF {query_vec} LIMIT 3
    """)
pd.set_option('display.max_colwidth', None)
funstuff = pd.DataFrame(hybrid_query)
funstuff

,product_purchased,ticket_opened,ticket_type,ticket_subject,ticket_description
0,Microsoft Office,2023-06-01 18:15:45,Billing inquiry,Network problem,"I've noticed a software bug in the Microsoft Office app. It's causing data loss and unexpected errors. How can I resolve this issue?\n\nThe default app version is 3.4. It's time to download and The issue I'm facing is intermittent. Sometimes it works fine, but other times it acts up unexpectedly."
1,Microsoft Office,2023-06-01 20:54:59,Refund request,Display issue,"I've encountered a data loss issue with my Microsoft Office. All the files and documents seem to have disappeared. Can you guide me on how to retrieve them? Also, are there any tools that will help prevent data loss? We I'm not sure if this issue is specific to my device or if others have reported similar problems."
2,Microsoft Office,2023-06-01 21:12:41,Cancellation request,Network problem,I've noticed a software bug in the Microsoft Office app. It's causing data loss and unexpected errors. How can I resolve this issue? We recommend that you start the installer with the version of Apple's mobile OS that you I've noticed a sudden decrease in battery life on my Microsoft Office. It used to last much longer.


In [11]:
# This block shows how to use Named Entity Recognition (NER) to do a combination
# text/term search with a vectorization of the same query
from transformers import pipeline
ner_tagger = pipeline("ner", aggregation_strategy="simple")
querystring = "I'm looking for a bunch of issues with Network"

queryvec = model.encode(querystring).tolist()
terms = ner_tagger(querystring)

print(terms)

searchterm = [term["word"] for term in terms][0] #Just pulling out the zeroth item
print(f"Extracted entity: {searchterm}")

ner_term_search_query = session.execute(
    f"""
    SELECT product_purchased, ticket_opened, ticket_type, ticket_subject, ticket_description
    FROM vsearch.vs_rca_learning
    WHERE ticket_description : '{searchterm}'
    ORDER BY embedding ANN OF {query_vec} LIMIT 3
    """)
res = pd.DataFrame(ner_term_search_query)

res

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'entity_group': 'ORG', 'score': 0.99121714, 'word': 'Network', 'start': 39, 'end': 46}]
Extracted entity: Network


,product_purchased,ticket_opened,ticket_type,ticket_subject,ticket_description
0,Microsoft Office,2023-06-01 07:52:06,Refund request,Product setup,"I've recently set up my Microsoft Office, but it fails to connect to any available networks. What steps should I take to troubleshoot this issue? I will provide an example in a future post.\n Administration of IP Add I'm using the original charger that came with my Microsoft Office, but it's not charging properly."
1,Microsoft Office,2023-06-01 13:04:49,Refund request,Peripheral compatibility,"I've recently set up my Microsoft Office, but it fails to connect to any available networks. What steps should I take to troubleshoot this issue?\n\nThe same goes for creating a new account for both the new account I've noticed a peculiar error message popping up on my Microsoft Office screen. It says '{error_message}'. What does it mean?"
2,Microsoft Surface,2023-06-01 02:35:30,Cancellation request,Network problem,"I've recently set up my Microsoft Surface, but it fails to connect to any available networks. What steps should I take to troubleshoot this issue?\n\n\nUpdate: In order to properly verify that your network connection fails and I need assistance as soon as possible because it's affecting my work and productivity."
